<a href="https://colab.research.google.com/github/he16946/Data_Labs/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv', delimiter=',')

In [14]:
#example of the returned array when using np.where
x = np.array([[1, 2, 3], [4, 5, 6]], np.int32)
np.where(x!=1)

(array([0, 0, 1, 1, 1]), array([1, 2, 0, 1, 2]))

In [23]:
array = df.values

def replace(array):

  #make a copy of data
  new_data = array.copy() 

  #return every non-99 index in the array
  rated = np.where(array!=99) 

  #how many values are non-99
  n_rated = len(rated[0]) 

  #returns index of a random 10% sample of non-99s
  idx = np.random.choice(n_rated, size=int(0.1*n_rated), replace = False) 

  #replaces the value at index of the random 10% and replaces them with 99 in the non-99 array
  new_data[rated[0][idx], rated[1][idx]] = 99

  #returns new array with a random 10% of 99s
  return new_data, (rated[0][idx], rated[1][idx])

new_array, idx = replace(array)

#original array value (where idx is an array of indexes)
array[idx[0][0], idx[1][0]] 

#new array value (where the idx index is the same position as in the original)
new_array[idx[0][0], idx[1][0]]

99.0